In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5794e00e9fb54e718553d972b368bf06 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=d1d261c71aa149c183e5045cbe223d44 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.93it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 40.32it/s]

Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/604d7d36fff7476f93f9b1168e94e104.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=a8f78edb804a46f48e8741d8da96cbdb in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_4941: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_4941
override True
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [10]:
request = client_low.requests[0]

In [11]:
code = client_low.code[0]

## Sync to high side

In [12]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

low_state

syft.service.sync.sync_state.SyncState

In [14]:
diff_state = compare_states(low_state, high_state)

diff_state

syft.service.sync.diff_state.DiffState

In [16]:
# syft absolute
# state.objs_to_sync
from syft.client.syncing import resolve

low_items_to_sync, high_items_to_sync = resolve(diff_state)

╭─ Request(92837aa1500a415586c893cd73fa875e) - State: NEW ────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ request_time = 2024-02-22 09:52:10                                                                   │  │
│  │  ‎ ‎ ‎ ‎ updated_at = None                                                                                    │  │
│  │  ‎ ‎ ‎ ‎ status = RequestStatus.PENDING                                                                       │  │
│  │  ‎ ‎ ‎ ‎ changes = [                                                                                          │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │  ‎ ‎ ‎ ‎ requesting_user_verify_key = 54215c682e0314ff5e74bc64ce662e7e888598e5f5bbd6ba7754dc1e437f866e        │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │ None                                                                                                      │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCode(8d5509b86c24446da725179693ff371a) - State: NEW ──────────────────────────────────────────────────╮
     │                                                                                                            │
     │  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ service_func_name = compute_mean                                                                │  │
     │  │  ‎ ‎ ‎ ‎ input_owners = [                                                                                │  │
     │  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ test_l                                                                                      │  │
     │  │  ‎ ‎ ‎ ‎ ]                                                                                               │  │
     │  │  ‎ ‎ ‎ ‎ code_status = [                                                                                 │  │
     │  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Node: test_l, Status: pending                                                               │  │
     │  │  ‎ ‎ ‎ ‎ ]                                                                                               │  │
     │  │  ‎ ‎ ‎ ‎ worker_pool_name = default-pool                                                                 │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │ None                                                                                                 │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                                            │
     ╰────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Do you want to keep the low state or the high state for these objects? choose 'low' or 'high'


 low


In [17]:
print("Low items to sync")
print(low_items_to_sync)
print()
print("High items to sync")
print(high_items_to_sync)

Low items to sync
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)

High items to sync
ResolvedSyncState(
  create_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode],
  update_objs=[],
  delete_objs=[]
)


In [18]:
client_low._sync_items(low_items_to_sync)

SyftSuccess: Synced 0 items

In [19]:
client_high._sync_items(high_items_to_sync)

SyftSuccess: Synced 2 items

# Run code high and sync back result

In [20]:
data_high = client_high.datasets[0].assets[0]

In [21]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = f59743e1d1ed454193d3355433800b72
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [22]:
# wait for the result
job_high.wait().get()

override True


22/02/24 11:54:25 FUNCTION LOG (f59743e1d1ed454193d3355433800b72): Computing mean...


17.0

In [23]:
job_info = job_high.info(public_metadata=True, result=True)

In [24]:
request = client_high.requests[0]
result_obj = job_high.result

print("result obj", result_obj.id)
accept_res = request.accept_by_depositing_result(job_info)

# request.approve()
# TODO: FIX THIS
request = client_high.requests[0]
code = request.code
log = job_high._get_log_objs()

result obj 4def7f503cea48d5a6c926b15d2f3567
Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: 4def7f503cea48d5a6c926b15d2f3567 as 54215c682e0314ff5e74bc64ce662e7e888598e5f5bbd6ba7754dc1e437f866e] 4def7f503cea48d5a6c926b15d2f3567
returning existing job
setting permission
None
None


In [25]:
action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [26]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

syft.service.sync.sync_state.SyncState

In [28]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.DiffState

In [29]:
low_items_to_sync, high_items_to_sync = resolve(diff_state_2, decision="high")

╭─ Request(92837aa1500a415586c893cd73fa875e) - State: DIFF ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─ Low side ────────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ updated_at=None,                                                                                     │  │
│  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ───────────────────────────────────────────────────────────────────────────────────────────────╮  │
│  │                                                                                                           │  │
│  │  class Request:                                                                                           │  │
│  │  ‎ ‎ ‎ ‎ updated_at=2024-02-22 09:54:32,                                                                      │  │
│  │  ‎ ‎ ‎ ‎ change=[                                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                                             │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ObjectMutation                                                      │  │
│  │  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange                                                   │  │
│  │  ‎ ‎ ‎ ‎ ]                                                                                                    │  │
│  │                                                                                                           │  │
│  ╰───────────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCode(8d5509b86c24446da725179693ff371a) - State: DIFF ─────────────────────────────────────────────────╮
     │                                                                                                            │
     │  ╭─ Low side ───────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ output_policy_state=b'',                                                                        │  │
     │  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.PENDING: 'pending'>, '')                                                │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ──────────────────────────────────────────────────────────────────────────────────────────╮  │
     │  │                                                                                                      │  │
     │  │  class UserCode:                                                                                     │  │
     │  │  ‎ ‎ ‎ ‎ output_policy_state=b'\x00\x00\x00\x00\xb9\x01\x00\x00\x00\x00\x00\x00...,                      │  │
     │  │  ‎ ‎ ‎ ‎ status=(<UserCodeStatus.APPROVED: 'approved'>, '')                                              │  │
     │  │                                                                                                      │  │
     │  ╰──────────────────────────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                                            │
     ╰────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Job(f59743e1d1ed454193d3355433800b72) - State: NEW ──────────────────────────────────────────────────╮
          │                                                                                                       │
          │  ╭─ Low side ──────────────────────────────────────────────────────────────────────────────────────╮  │
          │  │ None                                                                                            │  │
          │  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  │
          │  ╭─ High side ─────────────────────────────────────────────────────────────────────────────────────╮  │
          │  │                                                                                                 │  │
          │  │  class Job:                                                                                     │  │
          │  │  ‎ ‎ ‎ ‎ id = f59743e1d1ed454193d3355433800b72                                                      │  │
          │  │  ‎ ‎ ‎ ‎ result = 17.0                                                                              │  │
          │  │  ‎ ‎ ‎ ‎ resolved = True                                                                            │  │
          │  │  ‎ ‎ ‎ ‎ progress =                                                                                 │  │
          │  │  ‎ ‎ ‎ ‎ creation_time = 2024-02-22 11:54:23.275103                                                 │  │
          │  │                                                                                                 │  │
          │  ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯  │
          │                                                                                                       │
          ╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ SyftLog(1e77f1e114364b3ba0b441bd40ff1a8c) - State: NEW ─────────────────────────────────────────╮
               │                                                                                                  │
               │  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  │
               │  │ None                                                                                       │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  │
               │  │                                                                                            │  │
               │  │  class SyftLog:                                                                            │  │
               │  │  ‎ ‎ ‎ ‎ stdout = Computing mean...                                                            │  │
               │  │                                                                                            │  │
               │  │  ‎ ‎ ‎ ‎ stderr =                                                                              │  │
               │  │                                                                                            │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │                                                                                                  │
               ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ ActionObject(4def7f503cea48d5a6c926b15d2f3567) - State: NEW ────────────────────────────────────╮
               │                                                                                                  │
               │  ╭─ Low side ─────────────────────────────────────────────────────────────────────────────────╮  │
               │  │ None                                                                                       │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │  ╭─ High side ────────────────────────────────────────────────────────────────────────────────╮  │
               │  │                                                                                            │  │
               │  │  Pointer:                                                                                  │  │
               │  │  17.0                                                                                      │  │
               │  │                                                                                            │  │
               │  ╰────────────────────────────────────────────────────────────────────────────────────────────╯  │
               │                                                                                                  │
               ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from high side


In [31]:
print(high_items_to_sync)

ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)


In [32]:
client_low._sync_items(low_items_to_sync)

syncing obj with blob id 85cdd3d97244400e883bc0ad54affce1
Created 17.0 blob_id: 85cdd3d97244400e883bc0ad54affce1


SyftSuccess: Synced 5 items

In [33]:
client_high._sync_items(high_items_to_sync)

SyftSuccess: Synced 0 items

In [34]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [35]:
res_low = client_low_ds.code.compute_mean(data=data_low)

override False


In [36]:
res_low.get()

17.0

In [37]:
assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_policy.last_output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [38]:
private_high.mean()

17.0

In [39]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

8d5509b86c24446da725179693ff371a


SyftWarning: There are existing jobs for this user code, returning the latest one

In [40]:
job_low

```python
class Job:
    id: UID = f59743e1d1ed454193d3355433800b72
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Log 1e77f1e114364b3ba0b441bd40ff1a8c not available
JOB COMPLETED
    
```

In [41]:
job_low.wait().get()

17.0

In [42]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)